In [331]:
import numpy as np

# Variables

Helium viscosity from 2K to 5K: https://royalsocietypublishing.org/doi/10.1098/rspa.1950.0181

Gas helium viscosity: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392

Stainless steel thermal conductivity: https://thermtest.com/thermal-conductivity-of-steel

In [332]:
#From A2 in Ekin's book

thermalConductivityIntegral_300K = 3.06*10**3 #W/m

thermalConductivityIntegral_40K = 82.4 #W/m

thermalConductivityIntegral_4K = 0

In [460]:
#Fig 5 from https://cds.cern.ch/record/1973682/files/arXiv:1501.07100.pdf

thermalConductivityIntegral_1K = 0-(0.2+0.1+0.07)

#page 67 from Pobell

thermalConductivityIntegral_p1K = thermalConductivityIntegral_1K - 10**(-3)*0.5 - 4*10**(-4)*0.3 - 2*10**(-4)*0.1 #0.1K

In [487]:
OD_above_50K = 0.25*0.0254 #m
OD_below_50K = 0.125*0.0254 #m

thickness_above_50K = (0.25-0.18)*0.0254/2 #m, capillary wall thickness
thickness_below_50K = (0.125-0.085)*0.0254/2 #m, capillary wall thickness

length_above_50K = 10*0.0254 #m
length_below_50K = 4*20*0.0254 #m
length_4K_50K = 20*0.0254 #m

pressure_warm_end_condensation = 1*10**5 #Pa
pressure_cold_end_condensation = 0.25*10**5 #Pa, approximation
pressure_50K_condensation = 0.5*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

pressure_warm_end_evaporation = 0.01*10**5 #Pa
pressure_cold_end_evaporation = 0.25*10**5 #Pa, approximation
pressure_50K_evaporation = 0.05*10**5 #Pa, approximation. By ideal gas law: pressure_warm_end*42/300

viscosity_gas = 0.0196*10**(-3) #Pa*s, https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/#import-auto-id3073392
viscosity_liquid = 0.03*10**(-3) #Pa*s, https://royalsocietypublishing.org/doi/epdf/10.1098/rspa.1950.0181

stainless_steel_thermal_conductivity = 15*10**(-3) #W/K, https://thermtest.com/thermal-conductivity-of-steel

helium_volume = 0.1 #m^3, approximation

Poiseuille's Law: https://pressbooks.uiowa.edu/clonedbook/chapter/viscosity-and-laminar-flow-poiseuilles-law/

In [488]:
#radius is the inner radius
def getTubeVolmeFlowRate(viscosity, startPressure, endPressure, length, radius):
    R = 8*viscosity*length/(np.pi*(radius**4))
    return (startPressure-endPressure)/R

In [489]:
#thermalConductivity_low_temperature: integrated thermal conductivity from A2 in Ekin's book
def getHeatLoad(outerRadius, innerRadius, length, thermalConductivityIntegral_high_temperature, thermalConductivityIntegral_low_temperature):
    
    area = np.pi*(outerRadius**2-innerRadius**2)
    print(area)
    heat_load = (area/length)*(thermalConductivityIntegral_high_temperature-thermalConductivityIntegral_low_temperature)
    
    return heat_load #in W

In [490]:
#integrated
def getFilmHeatLoad(innerRadius, height):
    volume_flow = np.pi*3*10**(-6)*(2*innerRadius)**(3/4)*height**(-1/3) #m^3/s
    film_density = 150 #gram per m^3
    latent_heat = 25 #J per gram
    
    return volume_flow*film_density*latent_heat #W

# Gas throughput

## Condensation time

In [491]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
condensation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_warm_end_condensation,pressure_50K_condensation,length_above_50K,inner_radius_above_50K)
condensation_time_above_50K = helium_volume/condensation_volume_flow_rate_above_50K
print(condensation_volume_flow_rate_above_50K)
print(condensation_time_above_50K) #in seconds

0.10770726704988587
0.9284424601887248


In [492]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
condensation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_condensation,pressure_cold_end_condensation,length_4K_50K,inner_radius_below_50K)
condensation_time_below_50K = helium_volume/condensation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(condensation_time_below_50K) #in seconds

0.005355878159813628
74.68429789932321


The condensation time will be limited by the section below the 50K plate, which is 75 seconds.

# Heat Load (unit in W)

In [471]:
heat_flow_rate_above_50K = getHeatLoad(OD_above_50K/2, inner_radius_above_50K, length_above_50K, thermalConductivityIntegral_300K, thermalConductivityIntegral_40K)
print(heat_flow_rate_above_50K)

1.5251895120834685e-05
0.17879544453463525


In [472]:
heat_flow_rate_4K_50K = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_40K,thermalConductivityIntegral_4K)
print(heat_flow_rate_4K_50K)

4.2563428244189795e-06
0.0006903989148270156


In [473]:
heat_flow_rate_still_4K = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_4K,thermalConductivityIntegral_1K)
print(heat_flow_rate_still_4K)

4.2563428244189795e-06
3.1000922146358716e-06


In [494]:
heat_flow_rate_cold_still = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_1K,thermalConductivityIntegral_bottom)
print(heat_flow_rate_cold_still)

4.2563428244189795e-06
5.362321668559591e-09


In [493]:
getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/2, thermalConductivityIntegral_1K,thermalConductivityIntegral_bottom)

4.2563428244189795e-06


2.6811608342797953e-09

In [346]:
#total heat load in W
print(sum([heat_flow_rate_above_50K,heat_flow_rate_4K_50K,heat_flow_rate_still_4K+heat_flow_rate_bottom_still]))

0.17948894622283776


# Superfluid film heat load

In [424]:
print(inner_radius_below_50K)

0.0010795000000000002


In [439]:
getFilmHeatLoad((0.01/2)*0.0254, length_4K_50K)

8.911957604780256e-05

In [436]:
np.pi*3*10**(-6)*(0.01*0.0254)**(3/4)*length_4K_50K**(-1/3)

2.3765220279414013e-08

In [434]:
heat_flow_film_below_cold = getFilmHeatLoad(inner_radius_below_50K, length_4K_50K)
print(heat_flow_film_below_cold)

0.00044364675179058954


In [420]:
heat_flow_film_below_still = getFilmHeatLoad(inner_radius_below_50K, 2*length_4K_50K)

In [421]:
heat_flow_film_below_4K = getFilmHeatLoad(inner_radius_below_50K, 3*length_4K_50K)

In [422]:
print(heat_flow_film_below_4K)

0.0003076074771830429


In [423]:
print(heat_flow_film_below_still)

0.0003521226602473283


# Alternative radius choice for the section below 50 K

In [478]:
OD_below_50K = 0.0625*0.0254 #m
thickness_below_50K = (0.0625-0.0425)*0.0254/2 #m, capillary wall thickness

# Gas throughput

## Condensation time

In [479]:
inner_radius_above_50K = OD_above_50K/2 - thickness_above_50K
condensation_volume_flow_rate_above_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_warm_end_condensation,pressure_50K_condensation,length_above_50K,inner_radius_above_50K)
condensation_time_above_50K = helium_volume/condensation_volume_flow_rate_above_50K
print(condensation_volume_flow_rate_above_50K)
print(condensation_time_above_50K) #in seconds

0.10770726704988587
0.9284424601887248


In [480]:
inner_radius_below_50K = OD_below_50K/2 - thickness_below_50K
condensation_volume_flow_rate_below_50K = getTubeVolmeFlowRate(viscosity_gas,pressure_50K_condensation,pressure_cold_end_condensation,length_4K_50K,inner_radius_below_50K)
condensation_time_below_50K = helium_volume/condensation_volume_flow_rate_below_50K
print(volume_flow_rate_below_50K)
print(condensation_time_below_50K) #in seconds

0.005355878159813628
1194.9487663891714


The condensation time will be limited by the section below the 50K plate, which is 1195 seconds.

# Heat Load

In [481]:
heat_flow_rate_above_50K = getHeatLoad(OD_above_50K/2, inner_radius_above_50K, length_above_50K, thermalConductivityIntegral_300K, thermalConductivityIntegral_40K)
print(heat_flow_rate_above_50K)

1.5251895120834685e-05
0.17879544453463525


In [482]:
heat_flow_rate_4K_50K = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_40K,thermalConductivityIntegral_4K)
print(heat_flow_rate_4K_50K)

1.0640857061047449e-06
0.0001725997287067539


In [483]:
heat_flow_rate_still_4K = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_4K,thermalConductivityIntegral_1K)
print(heat_flow_rate_still_4K)

1.0640857061047449e-06
7.750230536589679e-07


In [484]:
heat_flow_rate_bottom_still = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/2, thermalConductivityIntegral_1K,thermalConductivityIntegral_bottom)
print(heat_flow_rate_bottom_still)

1.0640857061047449e-06
6.702902085699488e-10


In [486]:
heat_flow_rate_bottom_still = getHeatLoad(OD_below_50K/2, inner_radius_below_50K, length_below_50K/4, thermalConductivityIntegral_1K,thermalConductivityIntegral_bottom)
print(heat_flow_rate_bottom_still)

1.0640857061047449e-06
1.3405804171398977e-09


In [485]:
#total heat load in W
print(sum([heat_flow_rate_above_50K,heat_flow_rate_4K_50K,heat_flow_rate_still_4K+heat_flow_rate_bottom_still]))

0.17896881995668587


# Superfluid film heat load

In [443]:
heat_flow_film_below_cold = getFilmHeatLoad(inner_radius_below_50K, length_4K_50K)
print(heat_flow_film_below_cold)

0.0002637939368886076


In [444]:
heat_flow_film_below_still = getFilmHeatLoad(inner_radius_below_50K, 2*length_4K_50K)

In [445]:
heat_flow_film_below_4K = getFilmHeatLoad(inner_radius_below_50K, 3*length_4K_50K)

In [448]:
print(heat_flow_film_below_still)

0.00020937338645990428


In [449]:
print(heat_flow_film_below_4K)

0.0001829045002470559
